In [1]:
import pandas as pd
import json
from utils import gettingExplainerProperties

In [2]:
## Filter the explainers and their property values
def filter_properties(properties_dict, filter_properties_dict):
    for explainer, properties in properties_dict.items():
        for prop, prop_value in properties.items(): 
            # if isinstance(prop_value, list): # isinstance() check is performed to identify if the value is a list. 
            if prop_value[1] == "[":
                property_tmp_x = prop_value[:-2].replace('[','').split('], ')
                property_tmp = [list(x.replace("'","").split(', ')) for x in property_tmp_x]
                property_flatten = [x for y in property_tmp for x in y]
                filter_properties_dict.setdefault(prop, set()).update(property_flatten)
            elif prop_value[0] == "[":
                property_tmp = prop_value.replace('[','').replace(']','').replace("'","").split(', ')
                filter_properties_dict.setdefault(prop, set()).update(property_tmp) # update() method is used to add all elements of the list to the set       
            else:
                filter_properties_dict.setdefault(prop, set()).add(prop_value)

    return filter_properties_dict

In [3]:
# A function to get all the explainers in the BT, and to get all the properties that those explainers have
def get_all_properties_from_current_bt(original_case, explainer_data): 
    transformed_properties = {}
    
    nodes = original_case[0]['data']['trees'][0]['nodes']
    # print('nodes', nodes)   
       
    filtered_instances, instance_properties = {}, {}
    # Get all the explainers in the BT 
    for id, node_data in nodes.items():
        instance = node_data.get("Instance")
        # Extract "Instance" values of all IDs that start with '/'
        if instance and instance.startswith('/'):
            filtered_instances[id] = instance   
    explainer_list = list(explainer_data.keys())
    
    # Iterate through the filtered_instances
    for id, instance in filtered_instances.items():
        # print(f"ID: {id}, Instance: {instance}")
        # Find the explainer with a matching 'name' attribute
        matching_explainer = next((explainer for explainer in explainer_list if explainer == instance), None)
        #print('\nmatching_explainer', matching_explainer)
      
        # Check if a matching explainer was found
        if matching_explainer:            
            # Store the properties in the dictionary using the instance name as the key
            instance_properties[instance] = explainer_data[instance]

    # transformed_properties = filter_explainer_properties(instance_properties)
    transformed_properties = filter_properties(instance_properties, transformed_properties)
                                    
    return transformed_properties

In [4]:
def updateDict(my_dict):
    my_dict["explainer"] = my_dict.pop("Explainer")
    my_dict["technique"] = my_dict.pop("ExplainabilityTechniqueType")
    my_dict.pop("ExplainerDescription")
    my_dict["dataset_type"] = my_dict.pop("DatasetType")
    my_dict["explanation_type"] = my_dict.pop("ExplanationOutputType")
    my_dict.pop("ExplanationDescription")
    my_dict["concurrentness"] = my_dict.pop("Concurrentness")
    my_dict["portability"] = my_dict.pop("Portability")
    my_dict["scope"] = my_dict.pop("Scope")
    my_dict["target"] = my_dict.pop("TargetType")
    my_dict["presentations"] = my_dict.pop("OutputType")
    my_dict["computational_complexity"] = my_dict.pop("Complexity")
    my_dict["ai_methods"] = my_dict.pop("AIMethodType")
    my_dict["ai_tasks"] = my_dict.pop("AITaskType")
    my_dict["implementation"] = my_dict.pop("Backend")
    my_dict.pop("metadata")
    return my_dict

In [5]:
def getPropertiesFormat():
    properties = gettingExplainerProperties().to_dict(orient='records')
    data_dict = {item['Explainer']: item for item in properties}
    prop_dict = {x[0]: updateDict(x[1]) for x in data_dict.items()}
    return prop_dict

In [6]:
def getBTExplainerProperties(original_case):
    """
        This function is going to show the explainer properties that are in the current BT
        Petyo has to use this function to click all these properties in the form
    """
    
    return get_all_properties_from_current_bt(original_case, getPropertiesFormat())

In [7]:
with open("test_original.json", "r") as f:
    original_case = json.load(f)
    

In [8]:
print(getBTExplainerProperties(original_case))

{'explainer': {'/Tabular/LIME', '/Images/Anchors'}, 'technique': {'LIME', 'Simplification By Linear Regression', 'Anchor', 'Simplification By Rule Extraction', 'Simplification'}, 'dataset_type': {'Image', 'Multivariate'}, 'explanation_type': {'Anchor Explanation', 'Feature Influence Explanation'}, 'concurrentness': {'Post-hoc'}, 'portability': {'Model-agnostic'}, 'scope': {'Local'}, 'target': {'Prediction'}, 'presentations': {'figure', 'image', 'table', 'media', 'chart'}, 'computational_complexity': {'Quadratic time'}, 'ai_methods': {'Machine Learning'}, 'ai_tasks': {'Inductive Task', 'Regression', 'Classification'}, 'implementation': {'Sklearn', 'TensorFlow 2', 'TensorFlow 1', 'PyTorch', 'Any'}}


In [9]:
def getLibraryExplainerProperties():
    """
        This function returns all the properties for the explainers that we have in the explainer library
    """
    return filter_properties(getPropertiesFormat(), {})

In [10]:
getLibraryExplainerProperties()

{'explainer': {'/Images/Anchors',
  '/Images/Counterfactuals',
  '/Images/GradCam',
  '/Images/GradCamTorch',
  '/Images/IntegratedGradients',
  '/Images/LIME',
  '/Images/NearestNeighbours',
  '/Misc/AIModePerformance',
  '/Tabular/ALE',
  '/Tabular/Anchors',
  '/Tabular/DeepSHAPGlobal',
  '/Tabular/DeepSHAPLocal',
  '/Tabular/DicePrivate',
  '/Tabular/DicePublic',
  '/Tabular/DisCERN',
  '/Tabular/IREX',
  '/Tabular/Importance',
  '/Tabular/KernelSHAPGlobal',
  '/Tabular/KernelSHAPLocal',
  '/Tabular/LIME',
  '/Tabular/NICE',
  '/Tabular/TreeSHAPGlobal',
  '/Tabular/TreeSHAPLocal',
  '/Text/LIME',
  '/Text/NLPClassifier',
  '/Timeseries/CBRFox'},
 'technique': {'ALE',
  'Anchor',
  'Data-driven',
  'DiCE',
  'DisCERN',
  'Feature Relevance',
  'Game Theory Technique',
  'GradCam Technique',
  'Gradient-based Technique',
  'Influence Function',
  'Integrated Gradient Technique',
  'Knowledge Extraction',
  'LIME',
  'Optimisation Based',
  'SHAP',
  'Simplification',
  'Simplification

In [11]:
def get_all_properties_from_current_explainer(explainer, explainer_data):
    """
        This function returns the properties of the current explainer.
        Petyo should use this function on the form shown to include the criteria explainers when the 
        explainer reuse. The properties returned by this function have to be clicked in the form.
    """
    explainer_list = list(explainer_data.keys())
    result_properties, transformed_properties = {}, {}
    if explainer in explainer_list:
        result_properties[explainer] = explainer_data[explainer]
    transformed_properties = filter_properties(result_properties, transformed_properties)
                                    
    return transformed_properties

In [12]:
print(get_all_properties_from_current_explainer('/Images/Anchors', getPropertiesFormat()))
    

{'explainer': {'/Images/Anchors'}, 'technique': {'Simplification', 'Simplification By Rule Extraction', 'Anchor'}, 'dataset_type': {'Image'}, 'explanation_type': {'Anchor Explanation', 'Feature Influence Explanation'}, 'concurrentness': {'Post-hoc'}, 'portability': {'Model-agnostic'}, 'scope': {'Local'}, 'target': {'Prediction'}, 'presentations': {'figure', 'media', 'image'}, 'computational_complexity': {'Quadratic time'}, 'ai_methods': {'Machine Learning'}, 'ai_tasks': {'Inductive Task', 'Classification'}, 'implementation': {'Any'}}
